In [1]:
import re
import os
import numpy as np
from scipy.sparse import coo_matrix
from Bio import SeqIO
import tensorflow as tf
from time import time
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
import matplotlib.pyplot as plt

# 氨基酸序列的双联体ONE-HOT编码

In [2]:
alphabet = 'ACDEFGHIKLMNPQRSTVWYX'
daa = []
for x in alphabet:
    for y in alphabet:
        daa.append(x+y)

In [3]:
#把一个长度为L的氨基酸序列转换为一个矩阵型数组。按照20个氨基酸的两联体以ONE-HOT方式编码
#矩阵的列数是441
#矩阵的行数根据参数r，有：(L-1)+(L-2)+...+(L-r)=rL-(r+1)*r/2
def seq2DaaOneHotArray(sequence, r):
    L = len(sequence)
    N = r*L - ((r+1)*r)/2
    result = np.zeros(shape=(int(N), 441))
    m = 0
    for i in range(r):
        for j in range(L-i-1):
            aa = sequence[j]+sequence[j+i+1]
            k = daa.index(aa)
            result[m][k] = 1
            m = m + 1  
    return result

## 根据序列的fasta文件构建数据集

In [4]:
#读入序列文件和位点文件
def loadBindingsites(fastaFile, siteFile):
    #读序列文件，每一个序列构成字典的一项，
    #key：序列的id
    #value: 氨基酸序列的字母字符串
    data = {}
    for seq_record in SeqIO.parse(fastaFile, 'fasta'):
        data[seq_record.id] = seq_record.seq
    
    #读位点文件
    bindingsites = {}
    with open(siteFile, 'r') as pbsreader:
        i = 0
        for line in pbsreader:
            i = i + 1
            line = line.strip()
            if '>' in line:
                sid = line[1:]
            else:
                sites = line.split()
                bs = []
                for site in sites:
                    bs.append( int(site))
            if i%2 == 0:
                bindingsites[sid] = bs
      
    return (data, bindingsites)

In [5]:
data,bindingsites = loadBindingsites('../data/PDNA-224.fasta','../data/PDNA-224-binding-sites.txt')

## 把binding site和not binding site分别以稀疏矩阵存放在正负数据集中

In [6]:
#构建序列样本集
def splitDatasets(data, bindingsites,ws,r):
    positive=[]
    negative=[]
    for key in data:
        sites = bindingsites[key]
        p = data[key]
        seqlen = len(p)
        for j in range(seqlen):
            if j < ws:
                seq = str(p[j-ws:]) + str(p[0: ws+j+1])
            elif j > seqlen - ws -1:
                seq = str(p[j-ws:j]) + str(p[j:]) + str(p[0:ws-seqlen+j+1])
            else:
                seq = str(p[j-ws:j+ws+1])
            m = seq2DaaOneHotArray(seq, r)
            sm = coo_matrix(m)
            if j in sites:
                positive.append(sm)
            else:
                negative.append(sm)
    
    #positive = np.array(positive)
    #negative = np.array(negative)
    return (positive, negative)

In [7]:
positive,negative = splitDatasets(data, bindingsites,15,15)

In [20]:
type(positive)


list

# 定义CNN共享函数

In [ ]:
#定义权值
def weight(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.1), name='weight')

#定义偏置
def bias(shape):
    return tf.Variable(tf.constant(0.1, shape=shape), name='bias')

#定义卷积操作
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

#定义最大池化操作
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

#定义平均池化操作
def avg_pool_2x2(x):
    return tf.nn.avg_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')


# 定义CNN网络结构

In [ ]:
# x_train，x_test是scipy的coo_matrix稀疏矩阵对象
def cnn(x_train_pos, x_train_neg, x_test_pos, x_test_neg):
    x_train, y_train = shuffle(x_train, y_train)
    #输入层 345-by-441
    with tf.name_scope('input_layer'):
        x = tf.placeholder(tf.float32,shape=[None, 345, 441, 1], name='x')
    #第1个卷积层
    with tf.name_scope('conv_1'):
        W1 = weight([3,3,1,32])
        b1 = bias([32])
        conv_1 = conv2d(x, W1) + b1
        conv_2 = tf.nn.relu(conv_1)

    #第1个池化层 16-by-11
    with tf.name_scope('pool_1'):
        pool_1 = avg_pool_2x2(conv_1)

    #第2个卷积层
    with tf.name_scope('conv_2'):
        W2 = weight([3,3,32,64])
        b2 = bias([64])
        conv_2 = conv2d(pool_1, W2) + b2
        conv_2 = tf.nn.relu(conv_2)

    #第2个池化层 8-by-6
    with tf.name_scope("pool_2"):
        pool_2 = avg_pool_2x2(conv_2)

    #第3个卷积层
    with tf.name_scope('conv_3'):
        W3 = weight([3,3,64,128])
        b3 = bias([128])
        conv_3 = conv2d(pool_2, W3) + b3
        conv_3 = tf.nn.relu(conv_3)

    #第3个池化层 4-by-3
    with tf.name_scope('pool_3'):
        pool_3 = avg_pool_2x2(conv_3)

    #全连接层
    with tf.name_scope('fc'):
        #将最后一个池化层的128个通道的4-by-3的图像转换为一维向量，长度是128*4*3=1536
        W4 = weight([1536,256]) #全连接层定义256个神经元
        b4 = bias([256])
        flat = tf.reshape(pool_3, [-1, 1536])
        h = tf.nn.relu(tf.matmul(flat, W4)) + b4
        keep_prob = tf.placeholder(tf.float32)
        h_dropout = tf.nn.dropout(h, keep_prob)

    #输出层
    with tf.name_scope('output_layer'):
        W5 = weight([256,2])
        b5 = bias([2])
        pred = tf.nn.softmax(tf.matmul(h_dropout, W5) + b5)
    
    #构建网络模型
    with tf.name_scope("optimizer"):
        #定义占位符
        y = tf.placeholder(tf.int32, shape=[None, 2], name="label")
        #定义损失函数
        loss_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred,
                                                                                 labels=y))
        #选择优化器
        optimizer = tf.train.AdamOptimizer(learning_rate=0.003).minimize(loss_function)
    
    #定义准确率
    with tf.name_scope("evalulation"):
        correct_prediction = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
    #训练模型
    X_TRAIN_SIZE = len(x_train)
    TRAIN_EPOCHES = 25
    BATCH_SIZE = 50
    TOTAL_BATCH = int( np.ceil( X_TRAIN_SIZE / BATCH_SIZE))
    epoch= tf.Variable(0, name='epoch', trainable=False)
    STARTTIME = time()
    with tf.Session() as sess:
        init = tf.global_variables_initializer()
        sess.run(init)
        '''
        # 设置检查点存储目录
        ckpt_dir = "../log/"
        if not os.path.exists(ckpt_dir):
            os.makedirs(ckpt_dir)
        #生成saver
        saver = tf.train.Saver(max_to_keep=5)
        # 创建 summary_writer，用于写图文件
        summary_writer = tf.summary.FileWriter(ckpt_dir, sess.graph)
        # 如果有检查点文件，读取最新的检查点文件，恢复各种变量值
        ckpt = tf.train.latest_checkpoint(ckpt_dir )
        if ckpt != None:
            saver.restore(sess, ckpt)     
        else:
            print("Training from scratch.")

        start_epoch= sess.run(epoch)
        print("Training starts form {} epoch.".format(start_epoch+1))
        '''
        #迭代训练
        for ep in range(start_epoch, start_epoch + TRAIN_EPOCHES):
            for i in range(TOTAL_BATCH):
                start = (i * BATCH_SIZE) % X_TRAIN_SIZE
                end = min(start + BATCH_SIZE, X_TRAIN_SIZE)
                batch_x = x_train[start:end]
                batch_y = y_train[start:end]
                sess.run(optimizer,feed_dict={x: batch_x, y: batch_y, keep_prob:0.7})
                if i % 100 == 0:
                    print("Step {}".format(i), "finished")

            loss,acc = sess.run([loss_function,accuracy],feed_dict={x: batch_x, y: batch_y, keep_prob:0.7})
            
            print("Train epoch:", '%02d' % (sess.run(epoch)+1), \
                  "Loss=","{:.6f}".format(loss)," Accuracy=",acc)

            #保存检查点
            saver.save(sess,ckpt_dir+"DBPSite_cnn_model.cpkt",global_step=ep+1)

            sess.run(epoch.assign(ep+1))
    
        duration =time()-STARTTIME
        print("Train finished takes:",duration)   
    
        #计算测试集上的预测结果
        y_pred = sess.run(pred, feed_dict={x: x_test, y: y_test, keep_prob: 1.0})
        
    return y_pred

# 使用交叉验证

In [10]:
#对数据集X（list类型）按KFold划分训练集和测试集(list类型)
def splitByKFold(X, n=5):
    x_trains = []
    x_tests = []
    kf = KFold(n_splits=n)
    for train_index, test_index in kf.split(X):
        x_train, x_test = [],[]
        for k in train_index:
            x_train.append(X[k])
        for k in test_index:
            x_test.append(x[k])
        x_trains.append(x_train)
        x_tests.append(x_test)
     
    return x_trains, x_tests

#x_positive,x_negative都是list类型
def validationKFold(x_positive, x_negative, n=5):
    x_trains_pos, x_tests_pos = splitByKFold(x_positive, n)
    x_trains_neg, x_tests_neg = splitByKFold(x_negative, n)
    
    y_logists = np.ndarray((0,2))
    y_preds = np.ndarray((0,2))
    for i in range(n):            
        y_pred = cnn(x_trains_pos[i], x_trains_neg[i], x_tests_pos[i], x_tests_neg[i])
        
        k_test_pos = int( len( x_test_pos)) #正类测试样本数
        k_test_neg = int( len( x_test_neg)) #负类测试样本数
        y_test_pos = np.tile([1,0], (k_test_pos,1))
        y_test_neg = np.tile([0,1], (k_test_neg,1))
        
        y_test = np.append(y_test_pos, y_test_neg, axis=0)
        
        y_logists = np.append(y_logists, y_test, axis=0)
        y_preds = np.append(y_preds, y_pred, axis=0)
        
        return y_logists, y_preds

In [11]:
x_train, x_tests = splitByKFold(positive)

TypeError: only integer scalar arrays can be converted to a scalar index

In [18]:
x_trains = []
x_tests = []
X = positive[:20]
kf = KFold(n_splits=5)
for train_index, test_index in kf.split(X):
    print("*************")
    x_train = []
    for k in train_index:
        print(k)
        x_train.append(positive[k])
    x_trains.append(x_train)

*************
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
*************
0
1
2
3
8
9
10
11
12
13
14
15
16
17
18
19
*************
0
1
2
3
4
5
6
7
12
13
14
15
16
17
18
19
*************
0
1
2
3
4
5
6
7
8
9
10
11
16
17
18
19
*************
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [27]:
XX=[x_trains[1] x_trains[0]]

SyntaxError: invalid syntax (<ipython-input-27-e884af6036c0>, line 1)

In [26]:
x_trains[1]

[<345x441 sparse matrix of type '<class 'numpy.float64'>'
 	with 345 stored elements in COOrdinate format>,
 <345x441 sparse matrix of type '<class 'numpy.float64'>'
 	with 345 stored elements in COOrdinate format>,
 <345x441 sparse matrix of type '<class 'numpy.float64'>'
 	with 345 stored elements in COOrdinate format>,
 <345x441 sparse matrix of type '<class 'numpy.float64'>'
 	with 345 stored elements in COOrdinate format>,
 <345x441 sparse matrix of type '<class 'numpy.float64'>'
 	with 345 stored elements in COOrdinate format>,
 <345x441 sparse matrix of type '<class 'numpy.float64'>'
 	with 345 stored elements in COOrdinate format>,
 <345x441 sparse matrix of type '<class 'numpy.float64'>'
 	with 345 stored elements in COOrdinate format>,
 <345x441 sparse matrix of type '<class 'numpy.float64'>'
 	with 345 stored elements in COOrdinate format>,
 <345x441 sparse matrix of type '<class 'numpy.float64'>'
 	with 345 stored elements in COOrdinate format>,
 <345x441 sparse matrix of t